In [1]:
import numpy as np
import open3d as o3d
import laspy
from typing import Union, Tuple

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# 读取 PLY 文件
col_pcd = o3d.io.read_point_cloud("colmap_glacier.ply")
len(col_pcd.points)

520723

In [3]:
las_file = laspy.read("2016-11-28_AlphLake_SFM-Medium_USGSTransLOCAL.las")

# 提取 XYZ 坐标
las_points = np.vstack((las_file.x, las_file.y, las_file.z)).T

# 转换为 Open3D 点云对象
las_pcd = o3d.geometry.PointCloud()
las_pcd.points = o3d.utility.Vector3dVector(las_points)
len(las_pcd.points)

13178690

In [4]:
from pcd_reigstration import register_point_clouds_ndt

ModuleNotFoundError: No module named 'pclpy'

In [7]:
!pip install python-pcl

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      C:\Users\ShipM\AppData\Local\Temp\pip-install-x8828kxd\python-pcl_dafc42bd5a244f029cc30094b70fb64c\setup.py:51: SyntaxWarning: invalid escape sequence '\ '
        return stdout.decode().replace('\r\n', '').replace('\ ', ' ').replace('/', '\\').split(cut)
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\ShipM\AppData\Local\Temp\pip-install-x8828kxd\python-pcl_dafc42bd5a244f029cc30094b70fb64c\setup.py", line 4, in <module>
          from Cython.Distutils import build_ext
      ModuleNotFoundError: No module named 'Cython'
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> S

In [14]:
aligned_src, transform = register_point_clouds_ndt(col_pcd, las_pcd)

NameError: name 'register_point_clouds_ndt' is not defined